# LangChain: Agents

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

In [4]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [5]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Built-in LangChain tools

In [6]:
#!pip install -U wikipedia

In [3]:
from langchain_experimental.agents.agent_toolkits.base import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

ModuleNotFoundError: No module named 'langchain_experimental'

In [35]:
llm = ChatOpenAI(temperature = 0 , model = llm_model)
# define tools from load_tools
tools = load_tools(['llm-math','wikipedia'],llm=llm)
agent = initialize_agent(
        tools,
        llm,
        agent = AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        handle_parsing_errors=True,
        verbose = True
        )


In [37]:
agent(" tell me about trump, did he win the elections? ")



> Entering new AgentExecutor chain...
Thought: I should use Wikipedia to find information about Donald Trump and his election results.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Donald Trump"
}
```
Observation: Page: Donald Trump
Summary: Donald John Trump (born June 14, 1946) is an American politician, media personality, and businessman who served as the 45th president of the United States from 2017 to 2021. He won the 2024 election as the nominee of the Republican Party and is now the president-elect of the United States.  He is scheduled to begin his second term on January 20, 2025.   
Trump graduated with a bachelor's degree in economics from the University of Pennsylvania in 1968. After becoming president of the family real estate business in 1971, he renamed it the Trump Organization. After a series of bankruptcies in the 1990s, he launched side ventures, mostly licensing the Trump name. From 2004 to 2015, he produced and hosted the reality television series The 

Thought:Final Answer: Donald Trump won the 2024 United States presidential election as the Republican Party's nominee and is set to become the 47th president of the United States.

> Finished chain.


{'input': ' tell me about trump, did he win the elections? ',
 'output': "Donald Trump won the 2024 United States presidential election as the Republican Party's nominee and is set to become the 47th president of the United States."}

## Wikipedia example

In [20]:
question = "i want to tell me who is the author of attention is all you need paper"
result = agent(question) 



> Entering new AgentExecutor chain...
Question: Who is the author of the "Attention is All You Need" paper?
Thought: I should use Wikipedia to find the author of the paper.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Attention is All You Need"
}
```
Observation: Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer, based on the attention mechanism proposed in 2014 by Bahdanau et al. It is considered a foundational paper in modern artificial intelligence, as the transformer approach has become the main architecture of large language models like those based on GPT. At the time, the focus of the research was on improving Seq2seq techniques for machine translation, but the authors go further in the paper, foreseeing the technique's potential for other tasks like question answering

## Python Agent

In [21]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [25]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [26]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 



> Entering new AgentExecutor chain...
We can use the sorted() function in Python to sort the list of customers based on last name first and then first name.
Action: Python REPL
Action Input: customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))
print(sorted_customers)
Observation: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

Thought:The customers have been successfully sorted by last name and then first name.
Final Answer: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

> Finished chain.


"[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"

#### View detailed outputs of the chains

In [27]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nI

## Define your own tool

In [ ]:
#!pip install DateTime

In [28]:
from langchain.agents import tool
from datetime import date

In [38]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [39]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

**Note**: 

The agent will sometimes come to the wrong conclusion (agents are a work in progress!). 

If it does, please try running it again.

In [40]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")



> Entering new AgentExecutor chain...
Thought: I can use the `time` tool to find out today's date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2024-11-19
Thought:
Observation: Invalid or incomplete response
Thought:I will correct the previous action to get today's date properly.

Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Final Answer: 2024-11-19

> Finished chain.


Reminder: Download your notebook to you local computer to save your work.

In [ ]:
pip install langchain_experimental

In [ ]:
pip install langchain_community

In [ ]:
pip install --upgrade langchain-community


In [5]:
import langchain_community.utilities as utilities
print(dir(utilities))


['Any', 'REMOVED', 'TYPE_CHECKING', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__getattr__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_module_lookup', 'importlib']


In [10]:
from langchain_experimental.llm_bash.bash import BashProcess

In [1]:
from langchain_community.utilities.bash import BashProcess

bash = BashProcess(
    strip_newlines = False,
    return_err_output = False,
    persistent = False
)
bash.run("echo 'hello world'")

ModuleNotFoundError: No module named 'langchain_community.utilities.bash'

In [ ]:
from langchain_ollama.chat_models import _get_tool_calls_from_response as original_get_tool_calls
from uuid import uuid4

# Override _get_tool_calls_from_response
def _get_tool_calls_from_response(response):
    tool_calls = []
    if "message" in response:
        if "tool_calls" in response["message"] and response["message"]["tool_calls"] is not None:
            for tc in response["message"]["tool_calls"]:
                tool_calls.append(
                    tool_call(
                        id=str(uuid4()),
                        tool_name=tc.get("tool_name"),
                        args=tc.get("args"),
                    )
                )
    return tool_calls

# Monkey patch
import langchain_ollama.chat_models
langchain_ollama.chat_models._get_tool_calls_from_response = _get_tool_calls_from_response

from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.agents import load_tools
from langchain_experimental.llm_bash.bash import BashProcess
from langchain_ollama import ChatOllama



bash = BashProcess()

agent_executor = create_python_agent(
    llm = ChatOllama(model="llama3.1:latest", verbose=True),
    tool=PythonREPLTool(),
    handle_parsing_errors=True,
    verbose=True
)

agent_executor.invoke("""You have access to the terminal through the bash variable. open the terminal and perform ls command on terminal""")

In [10]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.1",
    temperature=0,
    # other params...
)

In [11]:

ai_msg = llm.invoke("Hey i'm saran - translate this sentence to french")
ai_msg

AIMessage(content='Bonjour Saran !\n\nThe translation of the sentence "translate this sentence to french" is:\n\n"Traduire cette phrase en français"\n\nOr, if you\'d like me to translate a specific sentence that you\'ve written (not just the instruction), please go ahead and share it with me!', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-11-27T03:23:46.243415Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2058956542, 'load_duration': 34024959, 'prompt_eval_count': 21, 'prompt_eval_duration': 284000000, 'eval_count': 58, 'eval_duration': 1739000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-27168c75-ca8a-46fa-a1d5-ee1c89e37575-0', usage_metadata={'input_tokens': 21, 'output_tokens': 58, 'total_tokens': 79})

In [6]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

In [8]:
from langchain_ollama.chat_models import _get_tool_calls_from_response as original_get_tool_calls
from uuid import uuid4

# Override _get_tool_calls_from_response
def _get_tool_calls_from_response(response):
    tool_calls = []
    if "message" in response:
        if "tool_calls" in response["message"] and response["message"]["tool_calls"] is not None:
            for tc in response["message"]["tool_calls"]:
                tool_calls.append(
                    tool_call(
                        id=str(uuid4()),
                        tool_name=tc.get("tool_name"),
                        args=tc.get("args"),
                    )
                )
    return tool_calls

# Monkey patch
import langchain_ollama.chat_models
langchain_ollama.chat_models._get_tool_calls_from_response = _get_tool_calls_from_response

In [9]:
from langchain.agents import AgentType, initialize_agent
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.1:latest", verbose=True)

shell_tool.description = shell_tool.description + f"args {shell_tool.args}".replace(
    "{", "{{"
).replace("}", "}}")
self_ask_with_search = initialize_agent(
    [shell_tool], llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)
self_ask_with_search.run(
    "Download the langchain.com webpage and grep for all urls. Return only a sorted list of them. Be sure to use double quotes."
)



> Entering new AgentExecutor chain...
Thought: I need to download the langchain.com webpage and then find all the URLs on it.
Action:
```
{
  "action": "terminal",
  "action_input": "curl -s 'https://langchain.com' > langchain.html; grep -oE 'https?://[^\" ]+' langchain.html | sort -u"
}
```
Executing command:
 curl -s 'https://langchain.com' > langchain.html; grep -oE 'https?://[^" ]+' langchain.html | sort -u


/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(



Observation: 
Thought:Action:
```
{
  "action": "terminal",
  "action_input": "curl https://langchain.com/langchain.html && cat langchain.html | grep -oE 'https?://[^\" ]+' | sort -u"
}
```

Executing command:
 curl https://langchain.com/langchain.html && cat langchain.html | grep -oE 'https?://[^" ]+' | sort -u


/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(



Observation:   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   166  100   166    0     0    792      0 --:--:-- --:--:-- --:--:--   794
<html>
<head><title>301 Moved Permanently</title></head>
<body>
<center><h1>301 Moved Permanently</h1></center>
<hr><center>openresty</center>
</body>
</html>

Thought:Thought: I now know that the website returned a 301 error, and I need to figure out how to download the webpage correctly.
Action:
```
{
  "action": "terminal",
  "action_input": "curl -s 'https://langchain.com' > langchain.html; grep -oE 'https?://[^\" ]+' langchain.html | sort -u"
}
```

Executing command:
 curl -s 'https://langchain.com' > langchain.html; grep -oE 'https?://[^" ]+' langchain.html | sort -u


...

/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(



Observation: 
Thought:Thought: I need to download the langchain.com webpage and then find all the URLs on it.
Action:
```
{
  "action": "terminal",
  "action_input": "curl -s 'https://langchain.com' > langchain.html; grep -oE '\"https?://[^\" ]+\"' langchain.html | sort -u"
}
```

Executing command:
 curl -s 'https://langchain.com' > langchain.html; grep -oE '"https?://[^" ]+"' langchain.html | sort -u


/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(



Observation: 
Thought:Thought: I need to download the langchain.com webpage and then find all the URLs on it.
Action:
```
{
  "action": "terminal",
  "action_input": "curl -s 'https://langchain.com' > langchain.html; grep -oE '\"https?://[^\" ]+\"' langchain.html | sort -u"
}
```



KeyboardInterrupt: 